### Packages

In [1]:
#在local cv 6377的基础上，加上MSE优化，MAE & MAE各自用N个seed 最终加权平均

In [2]:
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

### Input data

In [3]:
data_path = ''
train_data = pd.read_csv(data_path + 'train_dataset.csv')
test_data = pd.read_csv(data_path + 'test_dataset.csv')
sample_sub = pd.read_csv(data_path + 'submit_example.csv')

### Pre-processing

In [4]:
train_data.head(1)

,用户编码,用户实名制是否通过核实,用户年龄,是否大学生客户,是否黑名单客户,是否4G不健康客户,用户网龄（月）,用户最近一次缴费距今时长（月）,缴费用户最近一次缴费金额（元）,用户近6个月平均消费值（元）,...,当月是否景点游览,当月是否体育场馆消费,当月网购类应用使用次数,当月物流快递类应用使用次数,当月金融理财类应用使用总次数,当月视频播放类应用使用次数,当月飞机类应用使用次数,当月火车类应用使用次数,当月旅游资讯类应用使用次数,信用分
0,a4651f98c82948b186bdcdc8108381b4,1,44,0,0,0,186,1,99.8,163.86,...,1,1,713,0,2740,7145,0,0,30,664


In [5]:
#all chinese name- -
#rename one by one
print(train_data.columns)

Index(['用户编码', '用户实名制是否通过核实', '用户年龄', '是否大学生客户', '是否黑名单客户', '是否4G不健康客户',
       '用户网龄（月）', '用户最近一次缴费距今时长（月）', '缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）',
       '用户账单当月总费用（元）', '用户当月账户余额（元）', '缴费用户当前是否欠费缴费', '用户话费敏感度', '当月通话交往圈人数',
       '是否经常逛商场的人', '近三个月月均商场出现次数', '当月是否逛过福州仓山万达', '当月是否到过福州山姆会员店', '当月是否看电影',
       '当月是否景点游览', '当月是否体育场馆消费', '当月网购类应用使用次数', '当月物流快递类应用使用次数',
       '当月金融理财类应用使用总次数', '当月视频播放类应用使用次数', '当月飞机类应用使用次数', '当月火车类应用使用次数',
       '当月旅游资讯类应用使用次数', '信用分'],
      dtype='object')


In [6]:
# train_data.columns = ['uid','true_name_flag','age','uni_student_flag','blk_list_flag',\
#                      '4g_unhealth_flag','net_age_till_now','top_up_month_diff','top_up_amount',\
#                      'recent_6month_avg_use','total_account_fee','curr_month_balance',\
#                      'curr_overdue_flag','cost_sensitivity','connect_num','freq_shopping_flag',\
#                      'recent_3month_shopping_count','wanda_flag','sam_flag','movie_flag',\
#                      'tour_flag','sport_flag','online_shopping_count','express_count',\
#                      'finance_app_count','video_app_count','flight_count','train_count',\
#                      'tour_app_count','score']
# test_data.columns = train_data.columns[:-1]

### Feature Engineering

In [7]:
#age and net_age_in_month ---> 入网时的年龄 --- useless
#先前余额，当前余额 + 当月话费 - 上次缴费 --- useless
#充值金额/余额 --- useless
#当月话费/最近充值金额 --- useless
#六个月均值/充值金额 --- useless

#top up amount, 充值金额是整数，和小数，应该对应不同的充值途径？

def produce_offline_feat(train_data):
    train_data['不同充值途径']=0
    train_data['不同充值途径'][(train_data['缴费用户最近一次缴费金额（元）']%10==0)&train_data['缴费用户最近一次缴费金额（元）']!=0]=1

    return train_data

train_data = produce_offline_feat(train_data)
test_data = produce_offline_feat(test_data)

/home/eric/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
def produce_fee_rate(train_data):
    train_data['当前费用稳定性']=train_data['用户账单当月总费用（元）']/(train_data['用户近6个月平均消费值（元）']+1)
    
    ##当月话费/当月账户余额
    train_data['用户余额比例']=train_data['用户账单当月总费用（元）']/(train_data['用户当月账户余额（元）']+1)
    return train_data

train_data = produce_fee_rate(train_data)
test_data = produce_fee_rate(test_data)

In [9]:
#获取特征
def get_features(data):
    data.loc[data['用户年龄']==0,'用户年龄']=data['用户年龄'].mode()
    data['缴费金额是否能覆盖当月账单']=data['缴费用户最近一次缴费金额（元）']-data['用户账单当月总费用（元）']
    data['最近一次缴费是否超过平均消费额']=data['缴费用户最近一次缴费金额（元）']-data['用户近6个月平均消费值（元）']
    data['当月账单是否超过平均消费额']=data['用户账单当月总费用（元）']-data['用户近6个月平均消费值（元）']
    
    #映射年龄
    def map_age(x):
        if x<=18:
            return 1
        elif x<=30:
            return 2
        elif x<=35:
            return 3
        elif x<=45:
            return 4
        else:
            return 5
    data['是否大学生_黑名单']=data['是否大学生客户']+data['是否黑名单客户']
    data['是否去过高档商场']=data['当月是否到过福州山姆会员店']+data['当月是否逛过福州仓山万达']
    data['是否去过高档商场']=data['是否去过高档商场'].map(lambda x:1 if x>=1 else 0)
    data['是否_商场_电影']=data['是否去过高档商场']*data['当月是否看电影']
    data['是否_商场_体育馆']=data['是否去过高档商场']*data['当月是否体育场馆消费']
    data['是否_商场_旅游']=data['是否去过高档商场']*data['当月是否景点游览']
    data['是否_电影_体育馆']=data['当月是否看电影']*data['当月是否体育场馆消费']
    data['是否_电影_旅游']=data['当月是否看电影']*data['当月是否景点游览']
    data['是否_旅游_体育馆']=data['当月是否景点游览']*data['当月是否体育场馆消费']
    
    data['是否_商场_旅游_体育馆']=data['是否去过高档商场']*data['当月是否景点游览']*data['当月是否体育场馆消费']
    data['是否_商场_电影_体育馆']=data['是否去过高档商场']*data['当月是否看电影']*data['当月是否体育场馆消费']
    data['是否_商场_电影_旅游']=data['是否去过高档商场']*data['当月是否看电影']*data['当月是否景点游览']
    data['是否_体育馆_电影_旅游']=data['当月是否体育场馆消费']*data['当月是否看电影']*data['当月是否景点游览']
    
    data['是否_商场_体育馆_电影_旅游']=data['是否去过高档商场']*data['当月是否体育场馆消费']*data['当月是否看电影']*data['当月是否景点游览']
    
    discretize_features=['交通类应用使用次数','当月物流快递类应用使用次数','当月飞机类应用使用次数','当月火车类应用使用次数','当月旅游资讯类应用使用次数']
    data['交通类应用使用次数']=data['当月飞机类应用使用次数']+data['当月火车类应用使用次数']
    
    data['6个月平均占比总费用']=data['用户近6个月平均消费值（元）']/data['用户账单当月总费用（元）']+1
    
    data['用户账单当月总费用（零钱）'] = data['用户账单当月总费用（元）'].apply(lambda x:1 if(x-int(x))>0 else 0)
    data['用户账单当月总费用（整数）'] = data['用户账单当月总费用（元）'].apply(lambda x:int(x))
    
    data['6个月平均占比总费用']=data['用户近6个月平均消费值（元）']/data['用户账单当月总费用（元）']+1
    data['当月是否有使用理财应用'] = data['当月金融理财类应用使用总次数'].apply(lambda x:1 if x>0 else 0)
    data['当月是否有使用视频应用'] = data['当月视频播放类应用使用次数'].apply(lambda x:1 if x>0 else 0)
    data['当月是否使用网购类应用'] = data['当月网购类应用使用次数'].apply(lambda x:1 if x>0 else 0)
    data['最近三月是否有去过商场'] = data['近三个月月均商场出现次数'].apply(lambda x:1 if x>0 else 0)
    data['用户当月余额是否有余'] = data['用户当月账户余额（元）'].apply(lambda x:1 if x>0 else 0)
    data['当月是否使用过物流快递类应用'] = data['当月物流快递类应用使用次数'].apply(lambda x:1 if x>0 else 0)
    data['最近一次缴费金额是否为零'] = data['缴费用户最近一次缴费金额（元）'].apply(lambda x:1 if x==0 else 0)
    data['当月是否坐过飞机'] = data['当月飞机类应用使用次数'].apply(lambda x:1 if x>0 else 0)
    data['当月是否坐过火车'] = data['当月火车类应用使用次数'].apply(lambda x:1 if x>0 else 0)
    data['是否策划去旅游'] = data['当月旅游资讯类应用使用次数'].apply(lambda x:1 if x>0 else 0)
    data['用户年龄是否缺失'] = data['用户年龄'].apply(lambda x:1 if x==0 else x)
    data['正常客户'] = data['用户网龄（月）'].apply(lambda x:1 if x>12 else 0)
    del data['用户账单当月总费用（元）']
    def Start(x):
        if 24 > x['用户网龄（月）'] >=12:
            return 50
        elif 36 > x['用户网龄（月）'] >=24:
            return 150
        elif 48 > x['用户网龄（月）'] >=36:
            return 200
        elif 60 > x['用户网龄（月）'] >=48:
            return 250
        elif x['用户网龄（月）'] >=60:
            return 300
        else:
            return 0


    def Mon(x):
        if 50 >=x['用户近6个月平均消费值（元）'] > 20:
            return 20
        elif 80 >=x['用户近6个月平均消费值（元）'] > 50:
            return 50
        elif 120 >=x['用户近6个月平均消费值（元）'] > 80:
            return 100
        elif 200 >=x['用户近6个月平均消费值（元）'] > 120:
            return 150
        elif 400 >=x['用户近6个月平均消费值（元）'] > 200:
            return 200
        elif  x['用户近6个月平均消费值（元）'] > 400:
            return 250
        else:
            return 0

    def Level(x):
        a = x['缴费用户当前是否欠费缴费']
        b = x['得分']
#         if a:
#             b=b-100
        if 200>b>100:
            return 1
        elif 300>b>200:
            return 2
        elif 400>b>300:
            return 3
        elif 500>b>400:
            return 4
        elif b>500:
            return 5
        else:
            return 0
    data['网龄得分'] = data.apply(Start, axis=1)
    data['月均消费得分'] = data.apply(Mon, axis=1)
    data['得分'] = data['月均消费得分'] + data['网龄得分']
    data['等级'] = data.apply(Level, axis=1)
    del data['得分']
    del data['月均消费得分']
    del data['网龄得分']    
    def map_discretize(x):
        if x==0:
            return 0
        elif x<=5:
            return 1
        elif x<=15:
            return 2
        elif x<=50:
            return 3
        elif x<=100:
            return 4
        else:
            return 5
        
    for col in discretize_features[:]:
        data[col]=data[col].map(lambda x:map_discretize(x))
    
    return data

train_data=get_features(train_data)
test_data=get_features(test_data)

### Training

In [11]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.show()

In [12]:
#para
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    'metric': 'mae',
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'num_leaves': 31,
    'verbose': -1,
    'max_depth': 5,
    'lambda_l2': 5, 'lambda_l1': 0,'nthread': 8,
}

In [13]:
#para
params2 = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'mae',
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'num_leaves': 31,
    'verbose': -1,
    'max_depth': 5,
    'lambda_l2': 5, 'lambda_l1': 0,'nthread': 8,
    'seed': 89
}

In [14]:
cv_pred_all = 0
en_amount = 3
for seed in range(en_amount):
    NFOLDS = 5
    train_label = train_data['信用分']
    kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
    kf = kfold.split(train_data, train_label)

    train_data_use = train_data.drop(['用户编码','信用分','是否黑名单客户'], axis=1)
    test_data_use = test_data.drop(['用户编码','是否黑名单客户'], axis=1)


    cv_pred = np.zeros(test_data.shape[0])
    valid_best_l1_all = 0

    feature_importance_df = pd.DataFrame()
    count = 0
    for i, (train_fold, validate) in enumerate(kf):
        print('fold: ',i, ' training')
        X_train, X_validate, label_train, label_validate = \
        train_data_use.iloc[train_fold, :], train_data_use.iloc[validate, :], \
        train_label[train_fold], train_label[validate]
        dtrain = lgb.Dataset(X_train, label_train)
        dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=dvalid, verbose_eval=-1,early_stopping_rounds=50)
        cv_pred += bst.predict(test_data_use, num_iteration=bst.best_iteration)
        valid_best_l1_all += bst.best_score['valid_0']['l1']

#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = list(X_train.columns)
#         fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
#         fold_importance_df["fold"] = count + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        count += 1

    cv_pred /= NFOLDS
    valid_best_l1_all /= NFOLDS
    
    cv_pred_all += cv_pred
cv_pred_all /= en_amount
print('cv score for valid is: ', 1/(1+valid_best_l1_all))

/home/eric/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


fold:  0  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2412]	valid_0's l1: 14.7572
fold:  1  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2172]	valid_0's l1: 14.8379
fold:  2  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2642]	valid_0's l1: 14.712
fold:  3  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2294]	valid_0's l1: 14.6392
fold:  4  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[3666]	valid_0's l1: 14.4777


/home/eric/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


fold:  0  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[3027]	valid_0's l1: 14.7479
fold:  1  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[3506]	valid_0's l1: 14.8269
fold:  2  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2379]	valid_0's l1: 14.5598
fold:  3  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2655]	valid_0's l1: 14.5973
fold:  4  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2919]	valid_0's l1: 14.5987


/home/eric/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


fold:  0  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[3360]	valid_0's l1: 14.9402
fold:  1  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2433]	valid_0's l1: 14.729
fold:  2  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[3063]	valid_0's l1: 14.5215
fold:  3  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[3414]	valid_0's l1: 14.4592
fold:  4  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2533]	valid_0's l1: 14.688
cv score for valid is:  0.06382610738335795


In [15]:
cv_pred_all2 = 0
en_amount = 3
for seed in range(en_amount):
    NFOLDS = 5
    train_label = train_data['信用分']
    kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=(seed + 2019))
    kf = kfold.split(train_data, train_label)

    train_data_use = train_data.drop(['用户编码','信用分','是否黑名单客户'], axis=1)
    test_data_use = test_data.drop(['用户编码','是否黑名单客户'], axis=1)


    cv_pred = np.zeros(test_data.shape[0])
    valid_best_l2_all = 0

    feature_importance_df = pd.DataFrame()
    count = 0
    for i, (train_fold, validate) in enumerate(kf):
        print('fold: ',i, ' training')
        X_train, X_validate, label_train, label_validate = \
        train_data_use.iloc[train_fold, :], train_data_use.iloc[validate, :], \
        train_label[train_fold], train_label[validate]
        dtrain = lgb.Dataset(X_train, label_train)
        dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)
        bst = lgb.train(params2, dtrain, num_boost_round=10000, valid_sets=dvalid, verbose_eval=-1,early_stopping_rounds=50)
        cv_pred += bst.predict(test_data_use, num_iteration=bst.best_iteration)
        valid_best_l2_all += bst.best_score['valid_0']['l1']

#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = list(X_train.columns)
#         fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
#         fold_importance_df["fold"] = count + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        count += 1

    cv_pred /= NFOLDS
    valid_best_l2_all /= NFOLDS
    
    cv_pred_all2 += cv_pred
    
cv_pred_all2 /= en_amount
print('cv score for valid is: ', 1/(1+valid_best_l2_all))

fold:  0  training


/home/eric/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2102]	valid_0's l1: 14.7726
fold:  1  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2041]	valid_0's l1: 14.6954
fold:  2  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[1915]	valid_0's l1: 14.7859
fold:  3  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[1632]	valid_0's l1: 14.5127
fold:  4  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2455]	valid_0's l1: 14.7381


/home/eric/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


fold:  0  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2665]	valid_0's l1: 14.7345
fold:  1  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2924]	valid_0's l1: 14.7609
fold:  2  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2354]	valid_0's l1: 14.4969
fold:  3  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2406]	valid_0's l1: 14.5227
fold:  4  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2342]	valid_0's l1: 14.8165


/home/eric/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


fold:  0  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2342]	valid_0's l1: 14.5551
fold:  1  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2223]	valid_0's l1: 14.8393
fold:  2  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[1904]	valid_0's l1: 14.8145
fold:  3  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2220]	valid_0's l1: 14.6251
fold:  4  training
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[2048]	valid_0's l1: 14.6508
cv score for valid is:  0.06370665650604845


In [16]:
print('cv score for valid is: ', 1/(1+valid_best_l1_all))
print('cv score for valid is: ', 1/(1+valid_best_l2_all))

cv score for valid is:  0.06382610738335795
cv score for valid is:  0.06370665650604845


In [29]:
test_data_sub = test_data[['用户编码']]
test_data_sub['score'] = (cv_pred_all2 + cv_pred_all)/2
test_data_sub.columns = ['id','score']
test_data_sub['score1'] = cv_pred_all
test_data_sub['score2'] = cv_pred_all2

/home/eric/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/eric/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/eric/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

In [30]:
test_data_sub['score'] = test_data_sub['score'].apply(lambda x: int(np.round(x)))

/home/eric/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
test_data_sub[['id','score']].to_csv('19_03_18_1.csv',index=False)

In [ ]:
test_data1